In [14]:
#import dependecies
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')

import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

import numpy as np
import matplotlib.pyplot as plt 
from scipy import stats
import seaborn as sns; sns.set_theme(color_codes=True)
sns.set_context("paper")
%matplotlib inline

In [15]:
#create and connect to a sqlite3 db
conn = sqlite3.connect(r'/Users/rosaicelaroman/Desktop/portfolio_work/earthquake_modeling/Database/db/earthquake_db')

In [16]:
raw_data = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
raw_data

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.875000,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290000,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.119000,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460000,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20180,2022-01-07T20:32:10.944Z,60.011100,-148.479300,10.30,2.00,ml,NaN,NaN,NaN,0.3000,...,2022-02-01T23:02:58.793Z,"26 km WSW of Chenega, Alaska",earthquake,NaN,0.50,NaN,NaN,reviewed,ak,ak
20181,2022-01-07T20:30:37.581Z,57.714600,-155.765500,3.60,1.10,ml,NaN,NaN,NaN,0.5700,...,2022-02-01T23:02:58.585Z,"79 km W of Karluk, Alaska",earthquake,NaN,0.60,NaN,NaN,reviewed,ak,ak
20182,2022-01-07T20:29:18.910Z,38.826167,-122.805667,1.33,0.46,md,20.0,42.00,0.004417,0.0200,...,2022-01-12T08:32:10.153Z,"7km NW of The Geysers, CA",earthquake,0.16,0.41,0.053000,3.0,reviewed,nc,nc
20183,2022-01-07T20:24:43.614Z,64.980500,-147.361200,0.00,1.20,ml,NaN,NaN,NaN,0.2500,...,2022-02-01T23:02:59.043Z,"12 km ENE of Fox, Alaska",explosion,NaN,0.00,NaN,NaN,reviewed,ak,ak


In [17]:
#describe numerical data
raw_data.describe()

,latitude,longitude,depth,mag,nst,gap,dmin,rms,horizontalError,depthError,magError,magNst
count,20185.000000,20185.000000,20185.000000,20180.000000,14149.000000,16840.000000,13971.000000,20184.000000,14895.000000,20185.000000,15847.000000,16821.000000
mean,36.404868,-108.781447,22.853407,1.726416,20.488444,121.256412,0.714681,0.282825,1.898991,3.614683,0.288254,14.770109
std,19.838951,71.387189,52.951188,1.282050,15.480622,65.737282,2.251015,0.298190,3.259437,105.369952,0.552207,27.498336
min,-65.361900,-179.998500,-10.000000,-1.310000,2.000000,10.000000,0.000000,0.000000,0.090000,0.000000,0.000000,0.000000
25%,33.376667,-150.031100,3.420000,0.860000,9.000000,72.000000,0.021765,0.090000,0.290000,0.490000,0.104000,4.000000
50%,38.174100,-121.252667,8.820000,1.490000,16.000000,107.000000,0.066900,0.150000,0.490000,0.800000,0.160000,8.000000
75%,45.729500,-116.243700,18.600000,2.170000,27.000000,155.000000,0.204700,0.420000,1.150000,1.800000,0.242000,16.000000
max,85.309300,179.990100,653.870000,7.300000,189.000000,359.000000,39.750000,7.060000,54.500000,11327.700000,5.500000,619.000000


In [18]:
#describe categorical data
raw_data['type'].value_counts()

earthquake           19849
quarry blast           211
explosion               85
ice quake               27
other event             11
sonic boom               1
volcanic eruption        1
Name: type, dtype: int64

In [19]:
raw_data['place'].value_counts()

Rat Islands, Aleutian Islands, Alaska    247
8km NW of The Geysers, CA                236
7km NW of The Geysers, CA                169
off the coast of Oregon                  158
6km NW of The Geysers, CA                143
                                        ... 
50 km NNW of Sola, Vanuatu                 1
42 km ESE of Barkhan, Pakistan             1
9 km ESE of Clear, Alaska, Alaska          1
99 km SW of Atocha, Bolivia                1
12 km ENE of Fox, Alaska                   1
Name: place, Length: 9366, dtype: int64

In [20]:
raw_data['magType'].value_counts()

ml       11427
md        6389
mb        2022
mww        216
mb_lg       48
mwr         40
mw          20
mh          14
mlg          1
mlr          1
mwb          1
ms_20        1
Name: magType, dtype: int64

In [21]:
raw_data['id'].value_counts()

nn00831163      1
nn00832533      1
pr2022026004    1
ak02217a6huy    1
hv72890712      1
               ..
us6000gcsr      1
ak021fwi8p4b    1
us6000gct0      1
us6000gbpq      1
hv72861177      1
Name: id, Length: 20185, dtype: int64

In [22]:
raw_data['locationSource'].value_counts()

nc    3850
ak    3343
ci    3245
us    2692
hv    1905
nn    1522
av     758
uu     642
pr     537
mb     453
ok     432
uw     420
tx     295
nm      55
se      36
Name: locationSource, dtype: int64

In [23]:
raw_data['magSource'].value_counts()

nc     3850
ak     3344
ci     3245
us     2681
hv     1905
nn     1522
av      758
uu      642
pr      537
mb      453
ok      431
uw      420
tx      295
nm       55
se       36
guc      11
Name: magSource, dtype: int64

In [1]:
from dataprep.eda import plot
plot(raw_data)

ModuleNotFoundError: No module named 'dataprep'